# Assignment Problem - Assign Project to Employee

In [24]:
import pulp

model = pulp.LpProblem("Assignment Problem", pulp.LpMinimize)

workers = ["A", "B", "C"]
projects = ["1", "2", "3"]

cost = [[11, 14, 6], [8, 10, 11], [9, 12, 7]]
cost = {(workers[i], projects[j]): cost[i][j] for i in range(len(workers)) for j in range(len(projects))}

X = pulp.LpVariable.dicts("v", [(i, j) for i in workers for j in projects], cat='Binary')

model += pulp.lpSum(X[i, j] * cost[(i,j)] for i in workers for j in projects)

'''
We need to enforce that the total flow out of each worker node should be equal to 1. 
This represents the availability of each worker to be assigned to one project. 
'''
for i in workers:
    model += pulp.lpSum(X[i, j] for j in projects) == 1, f"{i} availability"

'''
We need to enforce that the total flow into each project node should be equal to 1.
This represents the requirement for one worker to be assigned to each project. T
'''
for j in projects:
    model += pulp.lpSum(X[i, j] for i in workers) == 1, f"project {j} requirement"

model.solve()

1

In [25]:
print("Status:", pulp.LpStatus[model.status])
print("Minimum total cost:", pulp.value(model.objective))

for i in workers:
    for j in projects:
        print(f"v_{i+j} = {X[i, j].value()}")


Status: Optimal
Minimum total cost: 25.0
v_A1 = 0.0
v_A2 = 0.0
v_A3 = 1.0
v_B1 = 0.0
v_B2 = 1.0
v_B3 = 0.0
v_C1 = 1.0
v_C2 = 0.0
v_C3 = 0.0


# Maximal Flow Model

In [28]:
import pulp

# Create the model
model = pulp.LpProblem("Maximal Flow", pulp.LpMaximize)

# Create the decision variables
flow = pulp.LpVariable.dicts("Flow", [("A", "B"), ("B", "C"), ("C", "A")], lowBound=0, cat='Continuous')

# Create the objective function
model += pulp.lpSum(flow[("A", "B")] + flow[("B", "C")] + flow[("C", "A")])

# Create the constraints
model += flow[("A", "B")] <= 2, "A_B_capacity"
model += flow[("B", "C")] <= 2, "B_C_capacity"
model += flow[("C", "A")] <= 2, "C_A_capacity"

model += flow[("A", "B")] == 1, "A_balance"
model += flow[("B", "C")] == 1, "B_balance"
model += flow[("C", "A")] == 1, "C_balance"

# Solve the model
model.solve()

# Print the results
print("Status:", pulp.LpStatus[model.status])
print("Maximum Flow:", pulp.value(model.objective))
for var in flow:
    print(f"Flow from {var[0]} to {var[1]}: {flow[var].value()}")

Status: Optimal
Maximum Flow: 3.0
Flow from A to B: 1.0
Flow from B to C: 1.0
Flow from C to A: 1.0


In [5]:
import pulp

# Define the model
model = pulp.LpProblem("Maximal Flow Model", pulp.LpMaximize)

# Define the decision variables
flow = {("1", "2"): pulp.LpVariable("Flow_1_2", 0, 3, pulp.LpInteger),
        ("1", "4"): pulp.LpVariable("Flow_1_4", 0, 2, pulp.LpInteger),
        ("1", "3"): pulp.LpVariable("Flow_1_3", 0, 10, pulp.LpInteger),
        ("2", "1"): pulp.LpVariable("Flow_2_1", 0, 1, pulp.LpInteger),
        ("2", "4"): pulp.LpVariable("Flow_2_4", 0, 1, pulp.LpInteger),
        ("2", "6"): pulp.LpVariable("Flow_2_6", 0, 2, pulp.LpInteger),
        ("3", "4"): pulp.LpVariable("Flow_3_4", 0, 3, pulp.LpInteger),
        ("3", "5"): pulp.LpVariable("Flow_3_5", 0, 2, pulp.LpInteger),
        ("4", "2"): pulp.LpVariable("Flow_4_2", 0, 1, pulp.LpInteger),
        ("4", "3"): pulp.LpVariable("Flow_4_3", 0, 1, pulp.LpInteger),
        ("4", "6"): pulp.LpVariable("Flow_4_6", 0, 1, pulp.LpInteger),
        ("5", "3"): pulp.LpVariable("Flow_5_3", 0, 1, pulp.LpInteger),
        ("5", "6"): pulp.LpVariable("Flow_5_6", 0, 6, pulp.LpInteger),
        ("6", "1"): pulp.LpVariable("Flow_6_1", 0, 1000, pulp.LpInteger),
        ("6", "2"): pulp.LpVariable("Flow_6_2", 0, 2, pulp.LpInteger),
        ("6", "4"): pulp.LpVariable("Flow_6_4", 0, 1, pulp.LpInteger)
        }

# Define the objective function
model += pulp.lpSum([flow[("6", "1")]]), "Maximal flow from node 1 to node 6"

'''
The flow balance constraints ensure that the flow into a node is equal to the flow out of a node for each node in the network. In other words, the total flow into a node must equal the total flow out of a node, excluding the source node and the sink node.

For example, for node i, the flow balance constraint can be expressed as:

flow_in_i = flow_out_i

where flow_in_i is the sum of all the flows into node i and flow_out_i is the sum of all the flows out of node i.

In the code, this can be written as follows:

model += flow[(j, i)] - flow[(i, j)] == 0, f"Node {i} balance"

where j and i are the nodes connected by the flow.
'''

# Define the flow balance constraints
model += flow[("6", "1")] + flow[("2", "1")] - flow[("1", "2")] - flow[("1", "4")] - flow[("1", "3")] == 0, "Node 1 balance"
model += flow[("1", "2")] + flow[("4", "2")] + flow[("6", "2")] - flow[("2", "1")] - flow[("2", "4")] - flow[("2", "6")] == 0, "Node 2 balance"
model += flow[("1", "3")] + flow[("4", "3")] + flow[("5", "3")] - flow[("3", "4")] - flow[("3", "5")] == 0, "Node 3 balance"
model += flow[("1", "4")] + flow[("2", "4")] + flow[("3", "4")] + flow[("6", "4")]- flow[("4", "3")] - flow[("4", "2")] - flow[("4", "6")] == 0, "Node 4 balance"
model += flow[("3", "5")] - flow[("5", "3")] - flow[("5", "6")] == 0, "Node 5 balance"
model += flow[("5", "6")] + flow[("4", "6")] + flow[("2", "6")]- flow[("6", "1")] - flow[("6", "2")] - flow[("6", "4")]== 0, "Node 6 balance"

# Solve the model
model.solve()

# Print the results
print("Maximal flow from node 1 to node 6: {}".format(pulp.value(model.objective)))
for edge, variable in flow.items():
    print("Flow from node {} to node {}: {}".format(edge[0], edge[1], variable.value()))


Maximal flow from node 1 to node 6: 5.0
Flow from node 1 to node 2: 2.0
Flow from node 1 to node 4: 1.0
Flow from node 1 to node 3: 2.0
Flow from node 2 to node 1: 0.0
Flow from node 2 to node 4: 0.0
Flow from node 2 to node 6: 2.0
Flow from node 3 to node 4: 0.0
Flow from node 3 to node 5: 2.0
Flow from node 4 to node 2: 0.0
Flow from node 4 to node 3: 0.0
Flow from node 4 to node 6: 1.0
Flow from node 5 to node 3: 0.0
Flow from node 5 to node 6: 2.0
Flow from node 6 to node 1: 5.0
Flow from node 6 to node 2: 0.0
Flow from node 6 to node 4: 0.0


# Shortest-Path Model

The objective function minimizes the total distance traveled by the flow between the nodes and the flow balance constraints ensure that the flow into a node is equal to the flow out of a node for each node in the network. The results will show the maximal flow from node 1 to node 6 and the flow from each node to another node.

In [17]:
import pulp

# Define the model
model = pulp.LpProblem("Shortest Path Model", pulp.LpMinimize)

# Define the decision variables
flow = {("1", "2"): pulp.LpVariable("Flow_1_2", 0, 100, pulp.LpInteger),
        ("2", "1"): pulp.LpVariable("Flow_2_1", 0, 100, pulp.LpInteger),
        ("1", "3"): pulp.LpVariable("Flow_1_3", 0, 200, pulp.LpInteger),
        ("3", "1"): pulp.LpVariable("Flow_3_1", 0, 200, pulp.LpInteger),
        ("2", "3"): pulp.LpVariable("Flow_2_3", 0, 50, pulp.LpInteger),
        ("3", "2"): pulp.LpVariable("Flow_3_2", 0, 50, pulp.LpInteger),
        ("2", "4"): pulp.LpVariable("Flow_2_4", 0, 200, pulp.LpInteger),
        ("4", "2"): pulp.LpVariable("Flow_4_2", 0, 200, pulp.LpInteger),
        ("2", "5"): pulp.LpVariable("Flow_2_5", 0, 100, pulp.LpInteger),
        ("5", "2"): pulp.LpVariable("Flow_5_2", 0, 100, pulp.LpInteger),
        ("3", "5"): pulp.LpVariable("Flow_3_5", 0, 40, pulp.LpInteger),
        ("5", "3"): pulp.LpVariable("Flow_5_3", 0, 40, pulp.LpInteger),
        ("4", "5"): pulp.LpVariable("Flow_4_5", 0, 150, pulp.LpInteger),
        ("5", "4"): pulp.LpVariable("Flow_5_4", 0, 150, pulp.LpInteger),
        ("4", "6"): pulp.LpVariable("Flow_4_6", 0, 100, pulp.LpInteger),
        ("6", "4"): pulp.LpVariable("Flow_6_4", 0, 100, pulp.LpInteger),
        ("5", "6"): pulp.LpVariable("Flow_5_6", 0, 100, pulp.LpInteger),
        ("6", "5"): pulp.LpVariable("Flow_6_5", 0, 100, pulp.LpInteger)
        }

'''
This objective function minimizes the total distance traveled by the flow between the nodes. 
The distance between each pair of nodes is multiplied by the flow variable representing the flow between these nodes.

model += pulp.lpSum([flow[(u, v)] * dist[(u, v)] for u, v in flow.keys()]), "Minimal Distance"
'''

# Define the objective function
model += pulp.lpSum([flow[("1", "2")] * 100, flow[("2", "1")] * 100, flow[("1", "3")] * 200,
flow[("3", "1")] * 200, flow[("2", "3")] * 50, flow[("3", "2")] * 50,
flow[("2", "4")] * 200, flow[("4", "2")] * 200, flow[("2", "5")] * 100,
flow[("5", "2")] * 100, flow[("3", "5")] * 40, flow[("5", "3")] * 40,
flow[("4", "5")] * 150, flow[("5", "4")] * 150, flow[("4", "6")] * 100,
flow[("6", "4")] * 100, flow[("5", "6")] * 100, flow[("6", "5")] * 100]), "Minimal Distance"

'''
The flow balance constraints ensure that the flow into a node is equal to the flow out of a node for each node in the network. In other words, the total flow into a node must equal the total flow out of a node, excluding the source node and the sink node.

For example, for node i, the flow balance constraint can be expressed as:

flow_in_i = flow_out_i

where flow_in_i is the sum of all the flows into node i and flow_out_i is the sum of all the flows out of node i.

In the code, this can be written as follows:

model += flow[(j, i)] - flow[(i, j)] == 0, f"Node {i} balance"

where j and i are the nodes connected by the flow.
'''
# Define the flow balance constraints
model += flow[("2", "1")] + flow[("3", "1")] - flow[("1", "2")] - flow[("1", "3")] == -1, "Supply of 1 unit in Node 1"
model += flow[("1", "2")] + flow[("3", "2")] + flow[("4", "2")] + flow[("5", "2")] - flow[("2", "1")] - flow[("2", "3")] - flow[("2", "4")] - flow[("2", "5")] == 0, "Node 2 balance"
model += flow[("1", "3")] + flow[("2", "3")] + flow[("5", "3")] - flow[("3", "1")] - flow[("3", "2")] -  flow[("3", "5")] == 0, "Node 3 balance"
model += flow[("2", "4")] + flow[("5", "4")] + flow[("6", "4")] - flow[("4", "2")] - flow[("4", "5")] - flow[("4", "6")] == 0, "Node 4 balance"
model += flow[("2", "5")] + flow[("3", "5")] +  flow[("4", "5")] +  flow[("6", "5")] -  flow[("5", "2")] - flow[("5", "3")] -  flow[("5", "4")] -  flow[("5", "6")] == 0, "Node 5 balance"
model += flow[("4", "6")] + flow[("5", "6")] - flow[("6", "4")] - flow[("6", "5")] == 1, "Demand of 1 unit in Node 6"

# Solve the model
model.solve()

1

In [18]:
# Print the results
print("Maximal flow from node 1 to node 6: {}".format(pulp.value(model.objective)))
for edge, variable in flow.items():
    print("Flow from node {} to node {}: {}".format(edge[0], edge[1], variable.value()))


Maximal flow from node 1 to node 6: 290.0
Flow from node 1 to node 2: 1.0
Flow from node 2 to node 1: 0.0
Flow from node 1 to node 3: 0.0
Flow from node 3 to node 1: 0.0
Flow from node 2 to node 3: 1.0
Flow from node 3 to node 2: 0.0
Flow from node 2 to node 4: 0.0
Flow from node 4 to node 2: 0.0
Flow from node 2 to node 5: 0.0
Flow from node 5 to node 2: 0.0
Flow from node 3 to node 5: 1.0
Flow from node 5 to node 3: 0.0
Flow from node 4 to node 5: 0.0
Flow from node 5 to node 4: 0.0
Flow from node 4 to node 6: 0.0
Flow from node 6 to node 4: 0.0
Flow from node 5 to node 6: 1.0
Flow from node 6 to node 5: 0.0
